## Libraries

In [ ]:
# /content/drive/MyDrive/IITB_EE/CS772_project/IEMOCAP.zip

In [ ]:
from IPython.display import Audio, display
import os
import pandas as pd
import re
import os
# import opensmile
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchaudio
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

**Load the IEMOCAP DATASET**

In [ ]:
!unzip '/content/drive/MyDrive/Assignments/CS772_project/IEMOCAP.zip'


In [ ]:
# Change directory to the root folder
os.chdir('/content/IEMOCAP')
# List the contents of the root folder
!ls

Documentation		  README.txt   Session1  Session3  Session5
iemocap_full_dataset.csv  README.txt~  Session2  Session4


### Audio data Extraction

In [ ]:
audio_file = 'Session5/sentences/wav/Ses05M_script02_1/Ses05M_script02_1_M030.wav'
Audio(audio_file)

**Loading The Metadata**

In [ ]:
meta_data = pd.read_csv('/content/IEMOCAP/iemocap_full_dataset.csv')
meta_data.head()

,session,method,gender,emotion,n_annotators,agreement,path
0,1,script,F,neu,3,3,Session1/sentences/wav/Ses01F_script02_1/Ses01...
1,1,script,F,fru,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...
2,1,script,F,xxx,0,0,Session1/sentences/wav/Ses01F_script02_1/Ses01...
3,1,script,F,sur,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...
4,1,script,F,neu,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...


In [ ]:
print(f'Size of the metadata = {meta_data.shape}')
meta_data.info()

Size of the metadata = (10039, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10039 entries, 0 to 10038
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   session       10039 non-null  int64 
 1   method        10039 non-null  object
 2   gender        10039 non-null  object
 3   emotion       10039 non-null  object
 4   n_annotators  10039 non-null  int64 
 5   agreement     10039 non-null  int64 
 6   path          10039 non-null  object
dtypes: int64(3), object(4)
memory usage: 549.1+ KB


**Confirming the correspondance between the audio samples available to that in the metadata by comparing the size of the samples availabe with that in the metadata**

In [ ]:
base_dir = '/content/IEMOCAP/'
sessions = [f'Session{i}' for i in range(1, 6)]

audio_files = []

for session in sessions:
    session_dir = os.path.join(base_dir, session)
    if os.path.exists(session_dir):
        wav_dir = os.path.join(session_dir, 'sentences', 'wav')
        for folder in os.listdir(wav_dir):
            folder_path = os.path.join(wav_dir, folder)
            for audio_file in os.listdir(folder_path):
                if audio_file.endswith('.wav'):
                    audio_files.append(os.path.join(folder_path, audio_file))

avail_samples = pd.DataFrame(audio_files, columns=['audio_path'])
# print(df.head())

In [ ]:
print('size of the available samples=',avail_samples.shape)

size of the available samples= (10039, 1)


### Text data Extraction

**Confirming the correspondance between the transcribed samples available to that in the metadata by comparing the size of the samples availabe with that in the metadata**

In [ ]:
base_dir = '/content/IEMOCAP/'
sessions = [f'Session{i}' for i in range(1, 6)]

audio_files = []

for session in sessions:
    session_dir = os.path.join(base_dir, session)
    if os.path.exists(session_dir):
        wav_dir = os.path.join(session_dir, 'sentences', 'ForcedAlignment')
        for folder in os.listdir(wav_dir):
            folder_path = os.path.join(wav_dir, folder)
            for audio_file in os.listdir(folder_path):
                if audio_file.endswith('.wdseg'):
                    audio_files.append(os.path.join(folder_path, audio_file))

avail_samples_trans = pd.DataFrame(audio_files, columns=['audio_path'])
print('size of the available samples=',avail_samples_trans.shape)

size of the available samples= (10037, 1)


**OBSERVATION**
1. For two of the audio we dont have the transcription. We will drop those samples

In [ ]:
drop_path1 =  'Session3/sentences/ForcedAlignment/Ses03M_impro03/Ses03M_impro03_M001.wdseg'
drop_path2 = 'Session3/sentences/ForcedAlignment/Ses03F_impro07/Ses03F_impro07_M030.wdseg'

**Function to read the transcribe text files**

In [ ]:
def read_wdseg(file_path):
    with open(file_path, 'r') as f:
        content = f.readlines()
    words = []
    start_time = []
    # print('content:',content)
    for line in content:
        if not line.strip().startswith(('SFrm', 'Total')):
            # print(line.strip())
            split_line = line.strip().split()
            # print(split_line)
            words.append(re.sub(r'\(\d+\)', '', split_line[-1]))
            start_time.append(split_line[0])
    # print(words,start_time)

    return ' '.join(words),' '.join(start_time)

In [ ]:
smap_id = 'Ses01F_impro01/Ses01F_impro01_F006'
audio_file = 'Session1/sentences/wav/'+smap_id+ '.wav'
display(Audio(audio_file))
words_file = 'Session1/sentences/ForcedAlignment/'+ smap_id + '.wdseg'
words,start_time = read_wdseg(words_file)
print(f'words = {words}, start time = {start_time}')


words = <s> WHAT <sil> I'M GETTING AN ID <sil> THIS IS WHY I'M HERE <sil> MY WALLET WAS STOLEN </s>, start time = 0 38 70 73 101 131 140 174 177 194 201 226 235 273 306 321 353 369 404


**Adding coulumns for path to the transcribed files, transcribed text and the token time stamps indormation for the samples to the metadata**

In [ ]:
meta_data_new = meta_data.copy()
def modify_path(path):
    parts = path.split('/')
    parts[2] = 'ForcedAlignment'
    audio_filename = parts[-1]
    transcript_filename = audio_filename.replace('.wav', '.wdseg')
    parts[-1] = transcript_filename
    return '/'.join(parts)

meta_data_new['transcript_path'] = meta_data_new['path'].apply(modify_path)
# Rename the 'path' column to 'audio_path'
meta_data_new.rename(columns={'path': 'audio_path'}, inplace=True)

# drop rows with Age equal to 35
meta_data_new = meta_data_new.drop(meta_data_new[meta_data_new.transcript_path == drop_path1].index)
meta_data_new = meta_data_new.drop(meta_data_new[meta_data_new.transcript_path == drop_path2].index)

# Apply the read_wdseg function to the 'transcript_path' column
meta_data_new['temp_result'] = meta_data_new['transcript_path'].apply(read_wdseg)

# Assign the results to the 'trans_words' and 'start_stamp' columns
meta_data_new[['trans_words', 'start_stamp']] = pd.DataFrame(meta_data_new['temp_result'].tolist(), index=meta_data_new.index)

# Drop the temporary 'temp_result' column
meta_data_new.drop(columns=['temp_result'], inplace=True)

meta_data_new.head()

,session,method,gender,emotion,n_annotators,agreement,audio_path,transcript_path,trans_words,start_stamp
0,1,script,F,neu,3,3,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Session1/sentences/ForcedAlignment/Ses01F_scri...,<s> FINE <sil> </s>,0 60 94 168
1,1,script,F,fru,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Session1/sentences/ForcedAlignment/Ses01F_scri...,<s> <sil> ++BREATHING++ </s>,0 93 135 140
2,1,script,F,xxx,0,0,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Session1/sentences/ForcedAlignment/Ses01F_scri...,<s> <sil> WHAT </s>,0 28 73 94
3,1,script,F,sur,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Session1/sentences/ForcedAlignment/Ses01F_scri...,<s> <sil> WHAT FLASHLIGHT <sil> </s>,0 27 37 56 111 142
4,1,script,F,neu,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Session1/sentences/ForcedAlignment/Ses01F_scri...,<s> <sil> THAT'S NOT YOUR FLASHLIGHT </s>,0 21 35 51 70 94 154


### Analysing The Emotion Categories available

In [ ]:
emo_cats = meta_data_new['emotion'].unique()
print('emo_cats', emo_cats)
value_counts = meta_data_new['emotion'].value_counts()
print(value_counts)

emo_cats ['neu' 'fru' 'xxx' 'sur' 'ang' 'hap' 'sad' 'exc' 'oth' 'fea' 'dis']
xxx    2506
fru    1849
neu    1708
ang    1103
sad    1084
exc    1040
hap     595
sur     107
fea      40
oth       3
dis       2
Name: emotion, dtype: int64


In [ ]:
agreement = meta_data_new['agreement'].unique()
print('agreement', agreement)
value_counts = meta_data_new['agreement'].value_counts()
print(value_counts)

agreement [3 2 0 4]
2    4466
3    2525
0    2506
4     540
Name: agreement, dtype: int64


**Obsservations:**
1. The 'xxx' tag in the emotion catfory corresponds to zero aggregment by the annotators. We can drop these samples.
2. We can also drop 'dis', 'oth', 'fea', 'sur' since they have very few samples
3. We will also drop 'fru' since it is not a very standardy emotion category and also our reference studies has omitted this label
4. Happiness category has relatively few samples so we can merge the happiness and excitment category sice the emotional characteristics are similia.We can name this category as 'hap'.
5. The original emotion categories can be merged to mitigate the unbalanced 
classes into four categories: anger, happiness(exitment+ happpiness), neutral, and sadness.

In [ ]:
# Define the categories to drop and the categories to merge
drop_cat = ['xxx', 'dis', 'oth', 'fea', 'sur', 'fru']
merge_cat = {'hap': ['hap', 'exc']}

# Drop rows with values in the 'emotion' column that are in the drop_cat list
meta_data_final = meta_data_new[~meta_data_new['emotion'].isin(drop_cat)].copy()

# Merge the emotion categories
for new_cat, old_cats in merge_cat.items():
    meta_data_final['emotion'] = meta_data_final['emotion'].replace(old_cats, new_cat)
# meta_data_final.head()

In [ ]:
emo_cats = meta_data_final['emotion'].unique()
value_counts = meta_data_final['emotion'].value_counts()
print('final data sample distribution\n',value_counts)

final data sample distribution
 neu    1708
hap    1635
ang    1103
sad    1084
Name: emotion, dtype: int64


In [ ]:
# concatenate the gender and session columns to get the spk label column
final_dataset = meta_data_final.copy()
final_dataset['spk'] = meta_data_final['gender'].astype(str) + meta_data_final['session'].astype(str)

In [ ]:
final_dataset.head()

,session,method,gender,emotion,n_annotators,agreement,audio_path,transcript_path,trans_words,start_stamp,spk
0,1,script,F,neu,3,3,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Session1/sentences/ForcedAlignment/Ses01F_scri...,<s> FINE <sil> </s>,0 60 94 168,F1
4,1,script,F,neu,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Session1/sentences/ForcedAlignment/Ses01F_scri...,<s> <sil> THAT'S NOT YOUR FLASHLIGHT </s>,0 21 35 51 70 94 154,F1
6,1,script,F,ang,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Session1/sentences/ForcedAlignment/Ses01F_scri...,<s> YOU KEEP SAYING MY FLASHLIGHT LIKE IT'S JU...,0 22 34 53 82 107 152 171 200 233 273,F1
7,1,script,F,ang,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Session1/sentences/ForcedAlignment/Ses01F_scri...,<s> HOW'S THAT SUPPOSED TO MAKE ME FEEL </s>,0 18 31 47 70 76 91 106 153,F1
8,1,script,F,neu,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Session1/sentences/ForcedAlignment/Ses01F_scri...,<s> <sil> YEAH <sil> </s>,0 22 36 72 126,F1


## Emotion Classification

**The dataset**

In [ ]:
emo_cats = final_dataset['emotion'].unique()
value_counts_emo = final_dataset['emotion'].value_counts()
print('\nfinal data sample distribution per emotion category\n',value_counts_emo)

speakers = final_dataset['spk'].unique()
value_counts_spk = final_dataset['spk'].value_counts()
print('\nfinal data sample distribution per speaker\n',value_counts_spk)


final data sample distribution per emotion category
 neu    1708
hap    1635
ang    1103
sad    1084
Name: emotion, dtype: int64

final data sample distribution per speaker
 F5    644
M3    614
M5    597
M1    578
F4    553
F3    536
M2    518
F1    507
F2    505
M4    478
Name: spk, dtype: int64


1. LOSO based cross validation. (leave one speaker out)
2. Audio only with output of the final RNN block used for classification
3. Audio only average pooling of the output of the RNN blocks used for classification
4. Audio only attention based weighted pooling of the output of the RNN blocks used for classification
5. Audio + word baased model

## Audio Feature Extraction

In [ ]:
#installing the oppine smile library
!pip install pandas opensmile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data = final_dataset.copy()

In [ ]:
data.shape

(5530, 11)

In [ ]:
# using the opensmile ComParE LLD features to get the frame level audio features
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)
print('number of features per frame = ',len(smile.feature_names))
print('name of the features = ',smile.feature_names)

number of features per frame =  65
name of the features =  ['F0final_sma', 'voicingFinalUnclipped_sma', 'jitterLocal_sma', 'jitterDDP_sma', 'shimmerLocal_sma', 'logHNR_sma', 'audspec_lengthL1norm_sma', 'audspecRasta_lengthL1norm_sma', 'pcm_RMSenergy_sma', 'pcm_zcr_sma', 'audSpec_Rfilt_sma[0]', 'audSpec_Rfilt_sma[1]', 'audSpec_Rfilt_sma[2]', 'audSpec_Rfilt_sma[3]', 'audSpec_Rfilt_sma[4]', 'audSpec_Rfilt_sma[5]', 'audSpec_Rfilt_sma[6]', 'audSpec_Rfilt_sma[7]', 'audSpec_Rfilt_sma[8]', 'audSpec_Rfilt_sma[9]', 'audSpec_Rfilt_sma[10]', 'audSpec_Rfilt_sma[11]', 'audSpec_Rfilt_sma[12]', 'audSpec_Rfilt_sma[13]', 'audSpec_Rfilt_sma[14]', 'audSpec_Rfilt_sma[15]', 'audSpec_Rfilt_sma[16]', 'audSpec_Rfilt_sma[17]', 'audSpec_Rfilt_sma[18]', 'audSpec_Rfilt_sma[19]', 'audSpec_Rfilt_sma[20]', 'audSpec_Rfilt_sma[21]', 'audSpec_Rfilt_sma[22]', 'audSpec_Rfilt_sma[23]', 'audSpec_Rfilt_sma[24]', 'audSpec_Rfilt_sma[25]', 'pcm_fftMag_fband250-650_sma', 'pcm_fftMag_fband1000-4000_sma', 'pcm_fftMag_spectralRollO

In [ ]:
# Extract audio features using OpenSMILE and storing them under 'feature' column
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)
def extract_lld_features(audio_path):
    return smile.process_file(audio_path)
data['features'] = data['audio_path'].apply(extract_lld_features)

In [ ]:
# displaying the set of LLD features of a given sample
samp_index = 0
samp_lld_features = data['features'][0]
print('size of the features (no. of frames in the sample X no. of features per frame)=',samp_lld_features.shape)
samp_lld_features.head()

size of the features (no. of frames in the sample X no. of features per frame)= (203, 65)


F0final_sma  \
file                                               start                  end                                   
Session1/sentences/wav/Ses01F_script02_1/Ses01F... 0 days 00:00:00        0 days 00:00:00.060000          0.0   
                                                   0 days 00:00:00.010000 0 days 00:00:00.070000          0.0   
                                                   0 days 00:00:00.020000 0 days 00:00:00.080000          0.0   
                                                   0 days 00:00:00.030000 0 days 00:00:00.090000          0.0   
                                                   0 days 00:00:00.040000 0 days 00:00:00.100000          0.0   

                                                                                                  voicingFinalUnclipped_sma  \
file                                               start                  end                                                 
Session1/sentences/wav/Ses01F_script02_1/Ses01F... 0 days 00:00:00        0 days 00:00:00.060000                   0.616906   
                                                   0 days 00:00:00.010000 0 days 00:00:00.070000                   0.613049   
                                                   0 days 00:00:00.020000 0 days 00:00:00.080000                   0.590242   
                                                   0 days 00:00:00.030000 0 days 00:00:00.090000                   0.567674   
                                                   0 days 00:00:00.040000 0 days 00:00:00.100000                   0.567427   

                                                                                                  jitterLocal_sma  \
file                                               start                  end                                       
Session1/sentences/wav/Ses01F_script02_1/Ses01F... 0 days 00:00:00        0 days 00:00:00.060000              0.0   
                                                   0 days 00:00:00.010000 0 days 00:00:00.070000              0.0   
                                                   0 days 00:00:00.020000 0 days 00:00:00.080000              0.0   
                                                   0 days 00:00:00.030000 0 days 00:00:00.090000              0.0   
                                                   0 days 00:00:00.040000 0 days 00:00:00.100000              0.0   

                                                                                                  jitterDDP_sma  \
file                                               start                  end                                     
Session1/sentences/wav/Ses01F_script02_1/Ses01F... 0 days 00:00:00        0 days 00:00:00.060000            0.0   
                                                   0 days 00:00:00.010000 0 days 00:00:00.070000            0.0   
                                                   0 days 00:00:00.020000 0 days 00:00:00.080000            0.0   
                                                   0 days 00:00:00.030000 0 days 00:00:00.090000            0.0   
                                                   0 days 00:00:00.040000 0 days 00:00:00.100000            0.0   

                                                                                                  shimmerLocal_sma  \
file                                               start                  end                                        
Session1/sentences/wav/Ses01F_script02_1/Ses01F... 0 days 00:00:00        0 days 00:00:00.060000               0.0   
                                                   0 days 00:00:00.010000 0 days 00:00:00.070000               0.0   
                                                   0 days 00:00:00.020000 0 days 00:00:00.080000               0.0   
                                                   0 days 00:00:00.030000 0 days 00:00:00.090000               0.0   
                                                   0 days 00:00:00.040000 0 days 0

In [ ]:
# example of a frame level input feature vecotor (size = 65)
lld_feature = np.array(data['features'][0].iloc[0,:])
print('size of the frame level feautre:',lld_feature.shape)
print('frame level feautre:',lld_feature)

size of the frame level feautre: (65,)
frame level feautre: [ 0.0000000e+00  6.1690617e-01  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -1.0000000e+02  1.7065620e-01  9.8834115e-01
  3.1472351e-03  1.2951387e-01  1.2921618e-01  2.7204838e-01
  4.1952792e-01  5.5444729e-01  6.6972893e-01  7.6589614e-01
  8.4702843e-01  9.1782922e-01  9.8224950e-01  1.0430994e+00
  1.1020993e+00  1.1600702e+00  1.2171308e+00  1.2728368e+00
  1.3262117e+00  1.3755809e+00  1.4180565e+00  1.4484811e+00
  1.4578775e+00  1.4327040e+00  1.3587462e+00  1.2323397e+00
  1.0691550e+00  8.9592761e-01  7.3424840e-01  5.9433109e-01
  1.7202210e-04  2.3886317e-04  1.4583333e+02  1.7708333e+02
  5.9375000e+02  2.3333333e+03  5.8162580e-03  6.9971338e+02
  4.8442259e+00  1.3393604e+06  2.4393411e+00  8.4248362e+00
 -2.2656319e-04  5.2607512e-01  2.7866819e-04  1.4346311e+01
 -2.2085631e+00  2.1203661e+01 -7.7395363e+00  2.7480936e+01
 -7.0422273e+00  1.2226391e+01 -5.6651120e+00  1.5888969e+01
 -1.0571196e+01  3.501309

## Further analysis of the Transcriptions

In [ ]:
trans_words = data['trans_words']

In [ ]:
import re

def display_non_text(data):
    excluded_tokens = {'<s>', '</s>', '<sil>', "'"}
    non_text_samples = []
    
    for text in data:
        # Remove excluded tokens from the text
        for token in excluded_tokens:
            text = text.replace(token, '')

        non_text = re.findall(r'[^a-zA-Z\s]', text)

        if non_text:
            non_text_samples.append((text, non_text))

    return non_text_samples

non_text_samples = display_non_text(trans_words)
count_breath = 0
count_laugh = 0
count_garb = 0
print('total samples:', len(non_text_samples))
for sample, non_text in non_text_samples:
    print(f"Sample: {sample}")
    print(f"Non-text characters: {non_text}")
    print()
    if sample==" ++BREATHING++ ":
      count_breath+=1
    elif sample== " ++LAUGHTER++ ":
      count_laugh+=1
    elif sample== " ++GARBAGE++ ":
      count_garb+=1
print(f"BREATHING: {count_breath},LAUGHTER:{count_laugh},GARBAGE:{count_garb}")

total samples: 162
Sample:  ++LAUGHTER++ 
Non-text characters: ['+', '+', '+', '+']

Sample:  THE PLANS YOU  MAKE  ++GARBAGE++ 
Non-text characters: ['+', '+', '+', '+']

Sample:  ++BREATHING++  UMM 
Non-text characters: ['+', '+', '+', '+']

Sample:   NO I MEAN IT JUST CAME ON REALLY QUICK YOU KNOW  I  MEAN IT WAS  A  ++GARBAGE++  IT WAS A SICKNESS  BUT IT JUST  
Non-text characters: ['+', '+', '+', '+']

Sample:  ++LAUGHTER++ FOR SOFTBALL  THATS  UNBELIEVABLE IS IT  SLOW  PITCH 
Non-text characters: ['+', '+', '+', '+']

Sample:   JUST WATCH IN  FIVE YEARS YOULL BE LIKE  ON THE  FORCE  YOU  CARRY AROUND  DONUTS IN YOUR BACK POCKET  ++LAUGHTER++  
Non-text characters: ['+', '+', '+', '+']

Sample:  PROFESSIONAL SOFTBALL PLAYER  MALIBU  ++LAUGHTER++ 
Non-text characters: ['+', '+', '+', '+']

Sample:  YEAH  WELL I DOUBT ++GARBAGE++  PING  PONG ++GARBAGE++ 
Non-text characters: ['+', '+', '+', '+', '+', '+', '+', '+']

Sample:  BUT YOU KNOW  IM  LIKE  ++GARBAGE++ 
Non-text characters: [

In [ ]:
idx = 1
audio_file = list(data[data['trans_words']== '<s> ++LAUGHTER++ </s>' ]['audio_path'])[idx]
Audio(audio_file)

**Drop the bad samples**

In [ ]:
data = data[(data['trans_words'] != '<s> ++GARBAGE++ </s>') & (data['trans_words'] != '<s> ++BREATHING++ </s>')].reset_index(drop=True)